In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-25 08:24:53--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-25 08:24:54 (75.4 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [2]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 34.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: gast
    Found existing installation: gast 0.6.0
    Uninstalling gast-0.6.0:
      

In [3]:
!pip install protobuf

In [2]:
from helper_functions import plot_loss_curves, walk_through_dir, unzip_data
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [3]:
dataset_list = tfds.list_builders()
print("food101" in dataset_list)

True


In [ ]:
# Load the dataset
dataset, info = tfds.load('food101', with_info=True, as_supervised=True, shuffle_files=True)




Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
#Splitting the dataset into training and validation sets
train_dataset, val_dataset = dataset['train'], dataset['validation']

In [ ]:
class_names = info.features["label"].names

###Exploring and becoming one with the data

In [ ]:
train_one_sample = train_dataset.take(1)

In [ ]:
for image, label in train_one_sample:
  print(f"""
  Image Shape: {image.shape}
  Image Dtype: {image.dtype}
  Image Label: {label}
  Class Name: {class_names[label.numpy()]}

  """)

In [ ]:
tf.reduce_min(image), tf.reduce_max(image)

In [ ]:
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis("off")

###CREATING PREPROCESSING FUNCTION FOR OUR DATA

In [ ]:
def preprocess_image(image, label, img_size=224):
  """
  *Resizing the image to be shape (224, 224)
  *Changing the datatype from uint8 -> float32
  """
  preprocessed_image = tf.image.resize(image, (img_size, img_size))

  return tf.cast(preprocessed_image, tf.float32), label

In [ ]:
preprocess_image(image, label)[0]

In [ ]:
train_dataset = train_dataset.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset = val_dataset.map(map_func=preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

val_dataset = val_dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#Creating a tensorboard callback
from helper_functions import create_tensorboard_callback

#Creating a ModelCheckpoint callback
from tensorflow.keras.callbacks import ModelCheckpoint

cp_path = "model_checkpoints.weights.h5"

model_checkpoint = ModelCheckpoint(cp_path,
                                   monitor="val_acc",
                                   save_best_only=True,
                                   save_weights_only=True)


###MIXED PRECISION TRAINING

In [ ]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")
mixed_precision.global_policy()

In [ ]:
#Model that supports mixed precision
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras import layers

IMG_SHAPE = (224 ,224 ,3)

base_model = EfficientNetV2B0(include_top=False)
base_model.trainable = False

inputs = layers.Input(IMG_SHAPE)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32)(x)

model = tf.keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  print(layer.name, layer.dtype, layer.dtype_policy)

In [ ]:
for layer in model.layers[1].layers:
  print(layer.name, layer.dtype, layer.dtype_policy)

In [ ]:
history_1 = model.fit(train_dataset, epochs=3, callbacks=[model_checkpoint, create_tensorboard_callback("tensor_hub_experiments.cp", "model_1")], validation_data=val_dataset)

In [ ]:
model.evaluate(val_data)